In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from keras.models import load_model
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt
import warnings

Model

In [ ]:
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='AUC',multi_label=True,num_labels=27,num_thresholds=498),
      keras.metrics.AUC(name='AUPR',curve='PR',multi_label=True,num_labels=27,num_thresholds=498), # precision-recall curve
]
def create_model(metrics=METRICS):
  model = Sequential([
    layers.Flatten(name="my_intermediate_layer"),
    layers.Dropout(0.4),  
    layers.Dense(1024,name="my_feature"),
    layers.ReLU(),
    layers.Dropout(0.4),
    layers.Dense(27, activation='sigmoid', name="my_last_layer"),
    ])
  return model

Data preprocessing

In [ ]:
#import data
y = pd.read_csv('./data/labels(2248).csv')
data = np.load('./data/gin_supervised_masking.npy')

In [ ]:
#split data
k_X_train,k_x_test,k_Y_train,k_y_test= train_test_split(data,y,test_size=0.1,random_state=2021)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau                     
checkpoint = ModelCheckpoint("./model/model_context2.hdf5",
                             monitor="val_accuracy",
                             mode="max",
                             save_best_only = True,
                             save_weights_only=False,
                             verbose=1)
earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0.0001, 
                          patience = 40,
                          verbose = 3,
                          mode = 'min',
                          restore_best_weights = True)
reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 10,
                              verbose = 3,
                              min_delta = 0.0001)
callbacks = [earlystop,reduce_lr]

train

In [ ]:
results=[]
time_start = time.time()
print(data1.shape)

kfolder = KFold(n_splits=10,shuffle=True,random_state=2021)
for i, (tra_id,val_id) in enumerate(kfolder.split(k_X_train,k_Y_train)):
  print(f"***********fold-{i+1}***********")

  c_x_train = k_X_train[tra_id]
  c_y_train = k_Y_train.iloc[tra_id]
  c_x_valid = k_X_train[val_id]
  c_y_valid = k_Y_train.iloc[val_id]
  model = create_model()
  opt = Adam(learning_rate=0.001)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=METRICS)
    
  # training
  history = model.fit(c_x_train,c_y_train,epochs=300,batch_size=128,
                      validation_data=(c_x_valid, c_y_valid.values),callbacks=callbacks,)

  baseline_results = model.evaluate(c_x_valid, c_y_valid.values, verbose=0)
  for name, value in zip(model.metrics_names, baseline_results):
    print(name, ': ', value)
  print()

  model.save_weights('./model/context/IADRMDL_masking_10fold'+str(i+1)+'.h5')

  result = np.array(baseline_results)
  results.append(result)



print("******************************************************************")

print(f"-Accuracy score_mean:{np.mean(np.array(results)[:,[1]])}")
print(f"-Accuracy score_mean:{np.std(np.array(results)[:,[1]])}")

print(f"-Precision score_mean:{np.mean(np.array(results)[:,[2]])}")
print(f"-Precision score_mean:{np.std(np.array(results)[:,[2]])}")

print(f"-Recall score_mean:{np.mean(np.array(results)[:,[3]])}")
print(f"-Recall score_mean:{np.std(np.array(results)[:,[3]])}")

print(f"-AUC score_mean:{np.mean(np.array(results)[:,[4]])}")
print(f"-AUC score_mean:{np.std(np.array(results)[:,[4]])}")

print(f"-AUPR score_mean:{np.mean(np.array(results)[:,[5]])}")
print(f"-AUPR score_mean:{np.std(np.array(results)[:,[5]])}")
   
time_end = time.time()
print(f"total running time: {(time_end - time_start)/60} minites")

gin_masking

In [ ]:
#load model
from keras.models import load_model
model_mask = create_model()
model_mask.build(k_X_train.shape)
model_mask.load_weights('./model/masking/IADRMDL_masking_10fold1.h5')
opt = Adam(learning_rate=0.001)
model_mask.compile(loss='binary_crossentropy', optimizer=opt, metrics=METRICS)
model_mask.fit(k_X_train,k_Y_train,batch_size=128, epochs=400)
model_mask.evaluate(k_x_test,k_y_test,return_dict=True)

In [ ]:
y_pre_masking = model_mask.predict(k_x_test)
np.save('./model/masking/masking_pre',y_pre_masking)

**gin_edge**

In [ ]:
#load model
from keras.models import load_model
model_edge = create_model()
model_edge.build(k_X_train.shape)
model_edge.load_weights('./model/edeg/IADRMDL_edeg_10fold2.h5')
opt = Adam(learning_rate=0.001)
model_edge.compile(loss='binary_crossentropy', optimizer=opt, metrics=METRICS)
model_edge.fit(k_X_train,k_Y_train,batch_size=128, epochs=300)
model_edge.evaluate(k_x_test,k_y_test,return_dict=True)

In [ ]:
y_pre_edge = model_edge.predict(k_x_test)
np.save('./model/edeg/edeg_pre',y_pre_edge)

gin_infomax

In [ ]:
#load model
from keras.models import load_model
model_infomax = create_model()
model_infomax.build(k_X_train.shape)
model_infomax.load_weights('./model/infomax/IADRMDL_infomax_10fold2.h5')
opt = Adam(learning_rate=0.001)
model_infomax.compile(loss='binary_crossentropy', optimizer=opt, metrics=METRICS)
model_infomax.fit(k_X_train,k_Y_train,batch_size=128, epochs=300)
model_infomax.evaluate(k_x_test,k_y_test,return_dict=True)
y_pre_infomax = model_infomax.predict(k_x_test)
np.save('./model/infomax/infomax_pre',y_pre_infomax)

gin_context

In [ ]:
#load model
from keras.models import load_model
model_context = create_model()
model_context.build(k_X_train.shape)
model_context.load_weights('./model/context/IADRMDL_context_10fold1.h5')
opt = Adam(learning_rate=0.001)
model_context.compile(loss='binary_crossentropy', optimizer=opt, metrics=METRICS)
model_context.fit(k_X_train,k_Y_train,batch_size=128, epochs=300)
model_context.evaluate(k_x_test,k_y_test,return_dict=True)
y_pre_context = model_context.predict(k_x_test)
np.save('./model/context/context_pre',y_pre_context)

attfp

In [ ]:
#load model
from keras.models import load_model
model_attfp = create_model()
model_attfp.build(k_X_train.shape)
model_attfp.load_weights('./model/attfp/IADRMDL_attfp_10fold1.h5')
opt = Adam(learning_rate=0.001)
model_attfp.compile(loss='binary_crossentropy', optimizer=opt, metrics=METRICS)
model_attfp.fit(k_X_train,k_Y_train,batch_size=128, epochs=300)
model_attfp.evaluate(k_x_test,k_y_test,return_dict=True)
y_pre_attfp = model_attfp.predict(k_x_test)
np.save('./model/attfp/attfp_pre',y_pre_attfp)

fp

In [ ]:
#load model
from keras.models import load_model
model_fp = create_model()
model_fp.build(k_X_train.shape)
model_fp.load_weights('./model/fp/IADRMDL_fp_10fold1.h5')
opt = Adam(learning_rate=0.001)
model_fp.compile(loss='binary_crossentropy', optimizer=opt, metrics=METRICS)
model_fp.fit(k_X_train,k_Y_train,batch_size=128, epochs=300)
model_fp.evaluate(k_x_test,k_y_test,return_dict=True)
y_pre_fp = model_attfp.predict(k_x_test)
np.save('./model/fp/fp_pre',y_pre_fp)

sub-models

In [ ]:
m1 = keras.metrics.BinaryAccuracy(name='accuracy',threshold=0.5)
m2 = keras.metrics.Precision(name='precision')
m3 = keras.metrics.Recall(name='recall')
m4 = keras.metrics.AUC(name='AUC',multi_label=True,num_labels=27,num_thresholds=498)
m5 = keras.metrics.AUC(name='AUPR',curve='PR',multi_label=True,num_labels=27,num_thresholds=498)

In [ ]:
#IADRDDI
y_pre1 = np.load('./model/masking/masking_pre.npy')
y_pre2 = np.load('./model/edeg/edeg_pre.npy')
y_pre3 = np.load('./model/info/info_pre.npy')
y_pre4 = np.load('./model/context/context_pre.npy')
y_pre5 = np.load('./model/fp/fp_pre.npy')
y_pre6 = np.load('./model/attfp/attfp_pre.npy')
y_pre = (y_pre1+y_pre2+y_pre4+y_pre5+y_pre6)/5